<a href="https://colab.research.google.com/github/vidhi-sys/Intelligent-Object-Recognition-and-Query-System-for-Warehouse-Robotics/blob/main/RAG_PART3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#pip install sentence-transformers faiss-cpu


This RAG system was developed to provide accurate, documentation-grounded responses for warehouse robotics operations. A synthetic knowledge base of 10 documents was created covering object handling procedures, safety protocols, equipment specifications, and troubleshooting guides.

Each document was divided into smaller chunks to improve retrieval granularity. Chunking ensures that semantically relevant sections are retrieved rather than entire documents, improving precision.

The system uses the SentenceTransformer model (all-MiniLM-L6-v2) to convert text into dense vector embeddings. These embeddings are stored in a FAISS index for efficient similarity search. When a user submits a query, it is embedded using the same model, and the top-k most similar chunks are retrieved using L2 distance.

The retrieved context is then used to generate a grounded response. Instead of relying solely on generative language modeling, the response is explicitly based on retrieved documentation, reducing hallucination risk and improving reliability in safety-critical robotic systems.

The architecture balances performance and simplicity. FAISS ensures low-latency retrieval, while transformer-based embeddings provide strong semantic understanding. Future improvements may include hybrid search (keyword + vector), more advanced response generation models, and integration with live robotic telemetry systems.

In [3]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# -------------------------------
# 1. KNOWLEDGE BASE (Synthetic Docs)
# -------------------------------

documents = [
"""
Fragile Item Handling:
Fragile items must be handled using reduced gripping force.
The robotic arm should operate at 60% normal speed.
Avoid sudden acceleration or rotation.
Use padded gripper attachments when available.
""",

"""
Heavy Object Handling:
Heavy objects require full gripper engagement.
Verify load balance before lifting.
Maximum lifting stability is achieved at slow vertical motion.
Do not exceed rated mechanical limits.
""",

"""
Hazardous Material Protocol:
Before handling hazardous materials, activate chemical safety mode.
Ensure environmental ventilation systems are active.
The robot must wear sealed gripping attachments.
Emergency stop must be within operator reach.
""",

"""
Gripper Arm Specifications:
The robotic gripper arm has a maximum weight capacity of 25 kilograms.
Maximum extension length is 1.2 meters.
Grip force ranges from 5N to 150N.
Operating temperature range is 0°C to 45°C.
""",

"""
Battery Maintenance Guide:
Battery levels must remain above 20% before heavy lifting tasks.
Recharge cycle duration is approximately 2 hours.
Do not operate under low voltage warning state.
""",

"""
Pallet Transport Instructions:
Pallets should be lifted from designated entry points.
Center alignment is required before vertical lift.
Tilt angle must not exceed 5 degrees.
""",

"""
Emergency Stop Procedure:
In emergency situations, press the red stop button immediately.
All motors will disengage.
System diagnostics will activate automatically.
""",

"""
Sensor Calibration Guide:
Cameras and depth sensors must be calibrated weekly.
Calibration improves object detection accuracy.
Misalignment may cause incorrect gripping positions.
""",

"""
Obstacle Avoidance Protocol:
The robot must scan surroundings before movement.
Maintain minimum 0.5 meter clearance from obstacles.
Reduce speed in high-density storage zones.
""",

"""
Troubleshooting Grip Failure:
If grip failure occurs, inspect for debris in gripper pads.
Verify hydraulic pressure levels.
Restart grip module controller if necessary.
"""
]

# -------------------------------
# 2. CHUNKING FUNCTION
# -------------------------------

def chunk_text(text, chunk_size=80):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size):
        chunks.append(" ".join(words[i:i+chunk_size]))
    return chunks

chunked_docs = []
for doc in documents:
    chunked_docs.extend(chunk_text(doc))

# -------------------------------
# 3. EMBEDDING MODEL
# -------------------------------

model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = model.encode(chunked_docs)

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# -------------------------------
# 4. RAG QUERY FUNCTION
# -------------------------------

def query_rag(query, top_k=3):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), top_k)

    retrieved_context = "\n\n".join([chunked_docs[i] for i in I[0]])

    # Simple grounded response generation
    response = f"""
Query: {query}

Retrieved Context:
{retrieved_context}

Generated Answer:
Based on the retrieved documentation, {retrieved_context}
"""

    return response


# -------------------------------
# 5. DEMO QUERIES
# -------------------------------

queries = [
    "How should the robot handle fragile items?",
    "What's the maximum weight capacity for gripper arm?",
    "What safety checks are needed before moving hazardous materials?",
    "What should be done if grip failure occurs?",
    "How should pallets be transported?"
]

for q in queries:
    print("="*80)
    print(query_rag(q))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


Query: How should the robot handle fragile items?

Retrieved Context:
Fragile Item Handling: Fragile items must be handled using reduced gripping force. The robotic arm should operate at 60% normal speed. Avoid sudden acceleration or rotation. Use padded gripper attachments when available.

Hazardous Material Protocol: Before handling hazardous materials, activate chemical safety mode. Ensure environmental ventilation systems are active. The robot must wear sealed gripping attachments. Emergency stop must be within operator reach.

Heavy Object Handling: Heavy objects require full gripper engagement. Verify load balance before lifting. Maximum lifting stability is achieved at slow vertical motion. Do not exceed rated mechanical limits.

Generated Answer:
Based on the retrieved documentation, Fragile Item Handling: Fragile items must be handled using reduced gripping force. The robotic arm should operate at 60% normal speed. Avoid sudden acceleration or rotation. Use padded gripper att